In [1]:
!pip install tensorflow-addons

In [2]:
import numpy as np
import csv
import time
import gc
import tensorflow as tf
from matplotlib import pyplot
from tensorflow import pad
from tensorflow.keras.datasets import cifar10, cifar100
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Dense, BatchNormalization, Normalization, Add
from tensorflow.keras.activations import relu
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import Input, Model
from tensorflow.keras.regularizers import l2
from tensorflow_addons.layers import StochasticDepth
from tensorflow.image import stateless_random_flip_left_right, stateless_random_crop
from tensorflow.random.experimental import stateless_split
from tensorflow.data import AUTOTUNE

# Load CIFAR-10 data.
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Initialize normalization layer.
norm_layer = Normalization(axis=(1, 2, 3), mean=X_train.mean(axis=0), variance=1)

# Convert from integers to floats.
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Apply normalization.
X_train = norm_layer(X_train)
X_test = norm_layer(X_test)

# Data augmentation.
# Pad X_train.
X_train = pad(X_train, [[0, 0], [4, 4], [4, 4], [0, 0]])

# Augmentation function.
def augment(image_label, seed):
    image, label = image_label
    
    # Make a new seed.
    new_seed = stateless_split(seed, num=1)[0, :]
    
    # Randomly flip and crop.
    image = stateless_random_flip_left_right(image, seed=seed)
    image = stateless_random_crop(image, size=[32, 32, 3], seed=new_seed)
    return image, label

rng = tf.random.Generator.from_seed(123, alg='philox')
# Wrapper function.
def f(x, y):
    seed = rng.make_seeds(2)[0]
    image, label = augment((x, y), seed)
    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(1280, reshuffle_each_iteration=True).map(f, num_parallel_calls=AUTOTUNE).batch(128).prefetch(AUTOTUNE)

path = 'cifar10/'
with open(path + 'results.csv', 'w') as results:
    writer = csv.writer(results)
    writer.writerow(['n', 'p_L', 'acc', 'time (s)', 'time (min)'])

def save_result(n, p_L, history, t, path):
    """Evaluate the model and store the results.

    :param n: Network size parameter.
    :param p_L: Final survival probability for linear decay.
    :param history: Model training history object.
    :param t: Training time.
    :param path: Path to results.
    """
    
    _, acc = model.evaluate(X_test, y_test, verbose=0)
    print('Test accuracy: %.3f' % (acc * 100.0))
    with open(path + 'results.csv', 'a') as results:
      writer = csv.writer(results)
      writer.writerow([n, p_L, round(100 * acc, 2), round(t), round(t / 60)])

    prefix = f"{path}raw/{n}_{int(100 * p_L)}_"

    train_loss_path = prefix + 'train_loss.npy'
    test_loss_path = prefix + 'test_loss.npy'
    train_acc_path = prefix + 'train_acc.npy'
    test_acc_path = prefix + 'test_acc.npy'

    with open(train_loss_path, 'wb') as f:
        np.save(f, history.history['loss'])

    with open(test_loss_path, 'wb') as f:
        np.save(f, history.history['val_loss'])

    with open(train_acc_path, 'wb') as f:
        np.save(f, history.history['accuracy'])

    with open(test_acc_path, 'wb') as f:
        np.save(f, history.history['val_accuracy'])

2023-05-08 20:32:30.144413: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-08 20:32:32.732590: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-08 20:32:32.732714: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/

In [3]:
def ResNet(n, n_out, p_L, weight_decay=1e-4, bottleneck=False):
  """ResNet for CIFAR-10 as described by He et al. (2015).

  This network has 2n + 2 layers.
  
  :param n: Network size parameter.
  :param n_out: Output size.
  :param p_L: Final survival probability for linear decay.
  :param weight_decay: Weight decay parameter.
  :param bottleneck: True if bottleneck blocks should be used.
  """

  stages = [16, 32, 64]
  inputs = Input(shape=(32, 32, 3))

  n_blocks = 3 * n
  block_count = 1
  
  # First convolution.
  x = Conv2D(16, 3, 1, padding='same', use_bias=False, kernel_initializer='he_normal', kernel_regularizer=l2(weight_decay))(inputs)
  x = BatchNormalization()(x)
  x = relu(x)
    
  # ResNet blocks.
  for i in range(len(stages)):
    n_filters = stages[i]
    for j in range(n):
      # Compute survival probability using linear decay.
      p_s = 1 - block_count / n_blocks * (1 - p_L)
        
      # Add ResNet block.
      if bottleneck:
        # Use bottleneck blocks.
        if i == 0 and j == 0:
          x = bottleneck_block(x, n_filters, p_s, weight_decay, first=True)
        elif j == 0:
          x = bottleneck_block(x, n_filters, p_s, weight_decay, downsample=True)
        else:
          x = bottleneck_block(x, n_filters, p_s, weight_decay)
      else:
        # Use classic ResNet blocks.
        if i > 0 and j == 0:
          x = block(x, n_filters, p_s, weight_decay, downsample=True)
        else:
          x = block(x, n_filters, p_s, weight_decay)
      block_count += 1
    
  # Pooling and dense output layer with softmax activation.
  x = GlobalAveragePooling2D()(x)
  outputs = Dense(n_out, activation='softmax', kernel_initializer='he_normal', kernel_regularizer=l2(weight_decay))(x)

  model = Model(inputs=inputs, outputs=outputs)

  # Compile model.
  opt = SGD(learning_rate=0.1, momentum=0.9)
  model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model


def block(x, n_filters, p_s, weight_decay, downsample=False, use_conv=False):
  """Classic ResNet block.
  
  :param x: Input.
  :param n_filters: Number of filters.
  :param p_s: Survival probability.
  :param weight_decay: Weight decay parameter.
  :param downsample: True if the layer should downsample.
  :param use_conv: True if a convolution operation should be used to match residual dimensions on downsampling.
  """
  
  if downsample:
    start_stride = 2
  else:
    start_stride = 1
    
  x_skip = x
    
  x = Conv2D(n_filters, 3, start_stride, padding='same', use_bias=False, kernel_initializer='he_normal', kernel_regularizer=l2(weight_decay))(x)
  x = BatchNormalization()(x)
  x = relu(x)
  x = Conv2D(n_filters, 3, 1, padding='same', use_bias=False, kernel_initializer='he_normal', kernel_regularizer=l2(weight_decay))(x)
  x = BatchNormalization()(x)

  if downsample:
    if use_conv:
      x_skip = Conv2D(n_filters, 1, 2, use_bias=False, kernel_initializer='he_normal', kernel_regularizer=l2(weight_decay))(x_skip)
      x_skip = BatchNormalization()(x_skip)
    else:
      x_skip = x_skip[:, ::2, ::2, :]
      missing = n_filters - x_skip.shape[3]
      x_skip = pad(x_skip, [[0, 0], [0, 0], [0, 0], [missing // 2, -(missing // -2)]])
  
  if p_s == 1:
    x = Add()([x_skip, x])
  else:
    x = StochasticDepth(p_s)([x_skip, x])
  x = relu(x)

  return x


def bottleneck_block(x, n_filters, p_s, weight_decay, first=False, downsample=False, use_conv=False):
  """Bottleneck ResNet block.
  
  :param x: Input.
  :param n_filters: Number of filters.
  :param p_s: Survival probability.
  :param weight_decay: Weight decay parameter.
  :param first: True if this is the first block.
  :param downsample: True if the layer should downsample.
  :param use_conv: True if a convolution operation should be used to match residual dimensions on downsampling.
  """
  
  if downsample:
    start_stride = 2
  else:
    start_stride = 1
    
  x_skip = x

  x = Conv2D(n_filters, 1, start_stride, kernel_initializer='he_normal', kernel_regularizer=l2(weight_decay))(x)
  x = BatchNormalization()(x)
  x = relu(x)
  x = Conv2D(n_filters, 3, 1, padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(weight_decay))(x)
  x = BatchNormalization()(x)
  x = relu(x)
  x = Conv2D(4 * n_filters, 1, 1, kernel_initializer='he_normal', kernel_regularizer=l2(weight_decay))(x)
  x = BatchNormalization()(x)

  if downsample:
    if use_conv:
      x_skip = Conv2D(4 * n_filters, 1, 2, kernel_initializer='he_normal', kernel_regularizer=l2(weight_decay))(x_skip)
      x_skip = BatchNormalization()(x_skip)
    else:
      x_skip = x_skip[:, ::2, ::2, :]
      missing = n_filters - x_skip.shape[3]
      x_skip = pad(x_skip, [[0, 0], [0, 0], [0, 0], [missing // 2, -(missing // -2)]])
  elif first:
    if use_conv:
      x_skip = Conv2D(4 * n_filters, 1, 1, kernel_initializer='he_normal', kernel_regularizer=l2(weight_decay))(x_skip)
      x_skip = BatchNormalization()(x_skip)
    else:
      missing = n_filters - x_skip.shape[3]
      x_skip = pad(x_skip, [[0, 0], [0, 0], [0, 0], [missing // 2, -(missing // -2)]])
  
  if p_s == 1:
    x = Add()([x_skip, x])
  else:
    x = StochasticDepth(p_s)([x_skip, x])
  x = relu(x)

  return x

In [4]:
epochs = 100

def scheduler(epoch, lr):
    if epoch == 82:
      return lr / 10
    else:
      return lr

In [5]:
# Dummy model to get accurate time measurements.
n = 2
n_out = 10
p_L = 0.9
model = ResNet(n, n_out, p_L)
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
history = model.fit(train_ds, epochs=1, validation_data=(X_test, y_test), callbacks=[callback], verbose=0)
tf.keras.backend.clear_session()
_ = gc.collect()

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-05-08 20:32:43.751415: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8200
2023-05-08 20:32:45.221649: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f2d2400ae60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-08 20:32:45.221693: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2023-05-08 20:32:45.294051: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [ ]:
# Run CIFAR-10 experiments.
n_out = 10
ns = [1, 3, 5, 7, 9]
p_Ls = [1, 0.95, 0.9, 0.85, 0.8]

for n in ns:
    for p_L in p_Ls:
        print(f"Running ResNet with n = {n} and p_L = {p_L}...")
        model = ResNet(n, n_out, p_L)
        callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
        t_start = time.time()
        history = model.fit(train_ds, epochs=epochs, validation_data=(X_test, y_test), callbacks=[callback], verbose=1)
        t = time.time() - t_start
        save_result(n, p_L, history, t, path)
        tf.keras.backend.clear_session()
        _ = gc.collect()
        print("")

Running ResNet with n = 1 and p_L = 1...
Epoch 1/100
391/391 [==============================] - 13s 30ms/step - loss: 1.6063 - accuracy: 0.4265 - val_loss: 1.5713 - val_accuracy: 0.4709 - lr: 0.1000
Epoch 2/100
391/391 [==============================] - 11s 28ms/step - loss: 1.1997 - accuracy: 0.5964 - val_loss: 1.2596 - val_accuracy: 0.5778 - lr: 0.1000
Epoch 3/100
391/391 [==============================] - 11s 28ms/step - loss: 1.0485 - accuracy: 0.6565 - val_loss: 1.1440 - val_accuracy: 0.6195 - lr: 0.1000
Epoch 4/100
391/391 [==============================] - 11s 27ms/step - loss: 0.9550 - accuracy: 0.6952 - val_loss: 1.2534 - val_accuracy: 0.6140 - lr: 0.1000
Epoch 5/100
391/391 [==============================] - 11s 27ms/step - loss: 0.8958 - accuracy: 0.7197 - val_loss: 1.0449 - val_accuracy: 0.6644 - lr: 0.1000
Epoch 6/100
391/391 [==============================] - 12s 30ms/step - loss: 0.8584 - accuracy: 0.7393 - val_loss: 1.1978 - val_accuracy: 0.6468 - lr: 0.1000
Epoch 7/100

In [ ]:
# Load CIFAR-100 data.
(X_train, y_train), (X_test, y_test) = cifar100.load_data()

# Initialize normalization layer.
norm_layer = Normalization(axis=(1, 2, 3), mean=X_train.mean(axis=0), variance=1)

# Convert from integers to floats.
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Apply normalization.
X_train = norm_layer(X_train)
X_test = norm_layer(X_test)

# Data augmentation.
# Pad X_train.
X_train = pad(X_train, [[0, 0], [4, 4], [4, 4], [0, 0]])

# Augmentation function.
def augment(image_label, seed):
    image, label = image_label
    
    # Make a new seed.
    new_seed = stateless_split(seed, num=1)[0, :]
    
    # Randomly flip and crop.
    image = stateless_random_flip_left_right(image, seed=seed)
    image = stateless_random_crop(image, size=[32, 32, 3], seed=new_seed)
    return image, label

rng = tf.random.Generator.from_seed(123, alg='philox')
# Wrapper function.
def f(x, y):
    seed = rng.make_seeds(2)[0]
    image, label = augment((x, y), seed)
    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(1280, reshuffle_each_iteration=True).map(f, num_parallel_calls=AUTOTUNE).batch(128).prefetch(AUTOTUNE)

path = 'cifar100/'
with open(path + 'results.csv', 'w') as results:
    writer = csv.writer(results)
    writer.writerow(['n', 'p_L', 'acc', 'time (s)', 'time (min)'])

In [ ]:
# Dummy model to get accurate time measurements.
n = 2
n_out = 100
p_L = 0.9
model = ResNet(n, n_out, p_L)
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
history = model.fit(train_ds, epochs=1, validation_data=(X_test, y_test), callbacks=[callback], verbose=0)
tf.keras.backend.clear_session()
_ = gc.collect()

In [ ]:
# Run CIFAR-100 experiments.
n_out = 100
ns = [1, 3, 5, 7, 9]
p_Ls = [1, 0.95, 0.9, 0.85, 0.8]

for n in ns:
    for p_L in p_Ls:
        print(f"Running ResNet with n = {n} and p_L = {p_L}...")
        model = ResNet(n, n_out, p_L)
        callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
        t_start = time.time()
        history = model.fit(train_ds, epochs=epochs, validation_data=(X_test, y_test), callbacks=[callback], verbose=1)
        t = time.time() - t_start
        save_result(n, p_L, history, t, path)
        tf.keras.backend.clear_session()
        _ = gc.collect()
        print("")